### Creating summaries of given csv file

In [1]:
import pandas as pd
import transformers
import torch

undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
    "ml6team/mbart-large-cc25-cnn-dailymail-nl-finetune"
)
tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
summarization_pipeline = transformers.pipeline(
    task="summarization",
    model=undisputed_best_model,
    tokenizer=tokenizer,
)
summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
    "nl_XX"
]


In [2]:
torch.cuda.is_available()


True

In [3]:
torch.cuda.device_count()


1

In [10]:
undisputed_best_model.to('cuda')

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
df=pd.read_csv("data_01.csv",sep="|")

In [ ]:
articles=df['nl_body']

In [ ]:
summary=[]
#Creating the summary with pretrained model and saving to a csv file in case an error and appending to dataframe 
counter=0
for text in articles:
    

    t= summarization_pipeline(
        text,
        do_sample=True,
        top_p=0.75,
        top_k=50,
        num_beams=4,
        min_length=50,
        early_stopping=True,
        truncation=True,
    )[0]["summary_text"]

    with open('summ.csv', 'a',encoding="utf-8-sig") as f:
    
        f.writelines(t)
        f.write('\n')

    counter=counter+1
    print(counter)

In [ ]:
df['nl_sum']=summary

In [ ]:
df.to_csv("data_02.csv",encoding="utf-8-sig",sep="|")